In [23]:
import pandas as pd
from utils.usda import USDA
from sklearn.linear_model import LinearRegression
import plotly.express as px

In [5]:
usda = USDA('IA','SOYBEANS')
ftr = usda.features()

Passing key to query params..
Connecting to http://quickstats.nass.usda.gov/api/api_GET/?
Preparing dataframe
Success!
Passing key to query params..
Connecting to http://quickstats.nass.usda.gov/api/api_GET/?
Preparing dataframe
Success!
Passing key to query params..
Connecting to http://quickstats.nass.usda.gov/api/api_GET/?
Preparing dataframe
Success!
Passing key to query params..
Connecting to http://quickstats.nass.usda.gov/api/api_GET/?
Preparing dataframe
Success!
Passing key to query params..
Connecting to http://quickstats.nass.usda.gov/api/api_GET/?
Preparing dataframe
Success!


In [38]:
ftr.keys()

Index(['countycd', 'countyname', 'statenbr', 'state_name', 'statecd', 'crop',
       'state_fips_code', 'year', 'areaharvested', 'areaplanted', 'production',
       'yield', 'totalacreloss', 'total_events', 'imdemnityamt', 'percipavg',
       'droughtavg', 'tempavg', 'shortdroughtavg', 'mintempavg', 'maxtempavg'],
      dtype='object')

In [46]:
#Production-Harvest Prediction

def predict_harvest(df):

    feature = pd.DataFrame()
    feature['production'] = df['production'].values
    feature['areaharvested'] = df['areaharvested'].values

    return feature
    #Production Feature


In [47]:
p = predict_harvest(ftr)

Exception: Data must be 1-dimensional

In [45]:
p.head()

,production,areaharvested
0,6769000,137800
1,7393000,132600
2,6043000,126500
3,3499000,79600
4,3743000,88400


In [37]:

fig = px.scatter(x=ftr['countycd'], y=ftr['totalacreloss'])
fig.show()

In [ ]:
gig